In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE180393'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Glomerular Transcriptome in the Cprobe Cohort"
!Series_summary	"We used microarrays to analyze the transcriptome of microdissected renal biopsies from patients with kidney disease and living donors. We derived pathway specific scores for Angiopoietin-Tie signaling pathway activation at mRNA level (or transcriptome level) for individual patients and studied the association of pathway activation with disease outcomes."
!Series_overall_design	"Glomerular gene expression data from micro-dissected human kidney biopsy samples  from patients with chronic kidney disease(Lupus, DN, IgA,HT, TN) and healthy living donors. Profiling was performed on Affymetrix ST2.1 microarray platform. "
Sample Characteristics Dictionary:
{0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset is likely to contain gene expression data
is_gene_available = True  # Based on the series summary, gene expression data is available

# Sample Characteristics Dictionary:
sample_characteristics_dict = {
    0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative GN', 'sample group: MN', 'sample group: AKI', 'sample group: LN-WHO V', 'sample group: FGGS', "sample group: 2'FSGS", 'sample group: Thin-BMD', 'sample group: Immuncomplex GN', 'sample group: LN-WHO-V', 'sample group: IgAN', 'sample group: LN-WHO IV+V', 'sample group: LN-WHO III+V', 'sample group: LN-WHO-I/II', 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 'sample group: CKD with mod-severe Interstitial fibrosis', 'sample group: Fibrillary GN', 'sample group: Interstitial nephritis', 'sample group: Hypertensive Nephrosclerosis', 'sample group: Unaffected parts of Tumor Nephrectomy'], 
    1: ['tissue: Glomeruli from kidney biopsy']
}

# For Lupus trait (Systemic Lupus Erythematosus), inferred from sample group
lupus_related_groups = {'LN-WHO III', 'LN-WHO IV', 'LN-WHO V', 'LN-WHO IV+V', 'LN-WHO III+V', 'LN-WHO-I/II', 'LN-WHO-V'}

trait_row = 0 if any(group for group in sample_characteristics_dict[0] if any(lg in group for lg in lupus_related_groups)) else None

# Age and gender information are not explicitly available in the sample characteristics dictionary
age_row = None
gender_row = None

# Data Type Conversion Functions

def convert_trait(value):
    """Convert the trait data to binary."""
    try:
        trait_value = value.split(":")[1].strip()  # Extract value after colon
        return 1 if any(lg in trait_value for lg in lupus_related_groups) else 0
    except Exception:
        return None

def convert_age(value):
    """Convert age data to continuous."""
    try:
        return float(value.split(":")[1].strip())  # Extract value after colon and convert to float
    except Exception:
        return None

def convert_gender(value):
    """Convert gender data to binary (0 for female, 1 for male)."""
    try:
        gender_value = value.split(":")[1].strip().lower()  # Extract value after colon and convert to lowercase
        if gender_value == 'male':
            return 1
        elif gender_value == 'female':
            return 0
        else:
            return None
    except Exception:
        return None

# Save cohort information
save_cohort_info('GSE180393', './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, extract clinical feature
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lupus_(Systemic_Lupus_Erythematosus)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lupus_(Systemic_Lupus_Erythematosus)/trait_data/GSE180393.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5607752': [0], 'GSM5607753': [0], 'GSM5607754': [0], 'GSM5607755': [0], 'GSM5607756': [0], 'GSM5607757': [0], 'GSM5607758': [0], 'GSM5607759': [0], 'GSM5607760': [0], 'GSM5607761': [0], 'GSM5607762': [0], 'GSM5607763': [1], 'GSM5607764': [0], 'GSM5607765': [0], 'GSM5607766': [1], 'GSM5607767': [0], 'GSM5607768': [0], 'GSM5607769': [0], 'GSM5607770': [0], 'GSM5607771': [0], 'GSM5607772': [0], 'GSM5607773': [1], 'GSM5607774': [0], 'GSM5607775': [1], 'GSM5607776': [1], 'GSM5607777': [0], 'GSM5607778': [0], 'GSM5607779': [1], 'GSM5607780': [0], 'GSM5607781': [0], 'GSM5607782': [0], 'GSM5607783': [0], 'GSM5607784': [0], 'GSM5607785': [1], 'GSM5607786': [0], 'GSM5607787': [1], 'GSM5607788': [1], 'GSM5607789': [0], 'GSM5607790': [0], 'GSM5607791': [1], 'GSM5607792': [1], 'GSM5607793': [1], 'GSM5607794': [0], 'GSM5607795': [0], 'GSM5607796': [1], 'GSM5607797': [0], 'GSM5607798': [1], 'GSM5607799': [0], 'GSM5607800': [0], 'GSM5607801': [0], 'GSM5607802': [0], 'GSM5607803': [0], 'GSM5607804

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the key names for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'ENTREZ_GENE_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression dataset
gene_data = apply_gene_mapping(gene_data, mapping_df)
